In [1]:
import numpy as np
import pandas as pd

In [2]:
path_us = 'data/'

i = 2020
cols_of_interest = ['quoteID', 'speaker', 'sentiment']

#Get us_data of 2020 year
df_full = pd.read_pickle(path_us + "us_" + str(i) + ".pkl.bz2",compression='bz2')[cols_of_interest]


In [3]:
'''
Format string to avoid to map same entity to two different entities
'''
#Static rules after analyzing the dataset
pres_dict_sur = {
    'george w. bush' : 'george bush',
    'george h.w. bush' : 'george bush',
    'donald j. trump' : 'donald trump',
    'donald trump jr.' : 'donald trump',
    'nicolás maduro guerra' : 'nicolas maduro',
    'nicolás maduro' : 'nicolas maduro'
    
}
pres_dict = {
  "trump": "donald trump",
  "obama": "barack obama",
  "clinton": "bill clinton",
"bush": 'george w. bush',
    'carter': 'jimmy carter',
    'moon' : 'moon jae-in',
    'maduro' : 'nicolas maduro'
  
}
def format_string(s):
    if s == None:
        return None
    a = s.lower()
    if ('president ' in a):
        a = a.replace('president ','')
        a = pres_dict.get(a, a)
       
    return pres_dict_sur.get(a, a)

In [4]:
df_temp = df_full.speaker.map(lambda x : format_string(x))
df_speak = df_full.copy()
df_speak['speaker'] = df_temp
df_speak

,quoteID,speaker,sentiment
0,2020-02-02-000235,danny davis,-0.787416
1,2020-03-29-000118,none,0.991827
2,2020-01-03-001237,julián castro,0.724682
3,2020-03-19-002801,laura ingraham,-0.996489
4,2020-02-28-003530,none,-0.769913
...,...,...,...
60889,2020-01-09-095402,donald trump,0.277540
60892,2020-01-24-095953,none,-0.606498
60893,2020-02-20-082429,none,-0.804123
60894,2020-02-24-068286,donald trump,0.996957


In [5]:
count_occ = df_speak.groupby('speaker').size()
df_count_occ = count_occ.to_frame().reset_index().rename(columns={0: "speaker_occ"})
df_count_occ.dtypes

speaker        object
speaker_occ     int64
dtype: object

In [6]:
df_speaker_occ = df_speak.set_index('speaker').join(df_count_occ.set_index('speaker'), on='speaker').reset_index()
df_modif = df_full.copy()
df_modif['speaker_occ'] = 1
df_modif['speaker'] = df_full.speaker.map(lambda x : format_string(x))
# df_speaker_occ =
final_df = pd.concat([df_speaker_occ,df_modif]).drop_duplicates('quoteID')
final_df['weighted_sentiment'] = final_df['sentiment']/final_df['speaker_occ']

In [7]:
none_ind = final_df['speaker'] == 'none'
final_df.loc[none_ind, 'weighted_sentiment'] = final_df[none_ind]['sentiment']
# final_df[none_ind]['weighted_sentiment'] = final_df[none_ind]['sentiment']* final_df[none_ind]['speaker_occ']

In [8]:
    name = "data/us_weighted_sent_" + str(i) + 'pkl.bz2'
    df_to_store = final_df[["quoteID", 'speaker', "speaker_occ", 'weighted_sentiment']]
    df_to_store = df_to_store.rename(columns={"speaker": "speaker_l"})
    
    df_to_store.to_pickle(name, compression='bz2')

In [9]:
df_to_store

,quoteID,speaker_l,speaker_occ,weighted_sentiment
0,2020-02-02-000235,danny davis,7,-0.112488
1,2020-03-29-000118,none,80902,0.991827
2,2020-01-03-001237,julián castro,51,0.014209
3,2020-03-19-002801,laura ingraham,167,-0.005967
4,2020-02-28-003530,none,80902,-0.769913
...,...,...,...,...
219942,2020-01-09-095402,donald trump,7308,0.000038
219943,2020-01-24-095953,none,80902,-0.606498
219944,2020-02-20-082429,none,80902,-0.804123
219945,2020-02-24-068286,donald trump,7308,0.000136
